# "Good Enough" vs Optimal: When Nearly Optimal Is Sufficient

This notebook demonstrates that **optimal solutions** are not always the only answer. Sometimes **"good enough"** solutions are sufficient, especially when considering implementation difficulty, robustness, or other practical factors.

Understanding this helps you:
- Recognize when optimality is necessary vs when "good enough" works
- Evaluate solutions beyond just objective value
- Make informed decisions about implementation tradeoffs
- Balance optimality with practicality


## Key Concepts

**Optimal Solution**:
- The best solution according to your objectives
- Found through systematic optimization
- May be difficult to implement or fragile

**"Good Enough" Solution**:
- Nearly optimal (close to best)
- May be easier to implement
- May be more robust to changes
- May be found faster

**Critical insight**: Optimal is best for objectives, but "good enough" may be best overall when considering implementation, robustness, and other practical factors.


## Scenario: Production Scheduling

You need to schedule production for 4 products. You want to minimize total cost while meeting demand.

**Decision**: How many units of each product to produce?

**Optimal solution**: Found through optimization (minimizes cost)

**"Good enough" solutions**: Nearly optimal but may have other advantages (easier implementation, more robust)


## Step 1: Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value


## Step 2: Set Up the Problem

Create a production scheduling problem:


In [ ]:
# Problem data
products = ['Product A', 'Product B', 'Product C', 'Product D']
demand = [100, 150, 120, 200]  # Demand for each product
costs = [10, 15, 12, 18]  # Cost per unit for each product
capacity = 600  # Total production capacity

print("PRODUCTION SCHEDULING PROBLEM:")
print("=" * 70)
problem_df = pd.DataFrame({
    'Product': products,
    'Demand': demand,
    'Cost per Unit': costs,
    'Total Cost if Meet Demand': [d * c for d, c in zip(demand, costs)]
})
display(problem_df)
print(f"\nTotal Capacity: {capacity} units")
print(f"Total Demand: {sum(demand)} units")


## Step 3: Find Optimal Solution

Use optimization to find the optimal solution:


In [ ]:
# OPTIMAL SOLUTION
model = LpProblem("Production_Optimal", LpMinimize)

# Decision variables: units to produce for each product
produce = [LpVariable(f"produce_{i}", lowBound=0, cat='Integer') for i in range(len(products))]

# Objective: Minimize total cost
model += lpSum([costs[i] * produce[i] for i in range(len(products))])

# Constraints: Meet demand and capacity
for i in range(len(products)):
    model += produce[i] >= demand[i], f"Demand_{products[i]}"

model += lpSum(produce) <= capacity, "Capacity_Limit"

# Solve
model.solve()

# Get optimal solution
optimal_solution = {}
optimal_cost = 0
for i in range(len(products)):
    units = value(produce[i])
    cost = costs[i] * units
    optimal_solution[products[i]] = {
        'Units': units,
        'Cost': cost
    }
    optimal_cost += cost

optimal_df = pd.DataFrame(optimal_solution).T
optimal_df.index.name = 'Product'

print("OPTIMAL SOLUTION:")
print("=" * 70)
display(optimal_df)
print(f"\nTotal Cost: ${optimal_cost:,.2f}")
print(f"Total Units: {optimal_df['Units'].sum():.0f}")


In [ ]:
# "GOOD ENOUGH" SOLUTIONS
# Solution 1: Simple heuristic - produce exactly demand (if capacity allows)
good_enough_1 = {}
cost_1 = 0
for i in range(len(products)):
    units = min(demand[i], capacity - sum([good_enough_1.get(products[j], 0) for j in range(i)]))
    if sum([good_enough_1.get(products[j], 0) for j in range(i+1)]) <= capacity:
        units = demand[i]
    else:
        units = max(0, capacity - sum([good_enough_1.get(products[j], 0) for j in range(i)]))
    good_enough_1[products[i]] = {
        'Units': units,
        'Cost': costs[i] * units,
        'Reason': 'Simple: meet demand in order'
    }
    cost_1 += costs[i] * units

# Solution 2: Prioritize low-cost products
sorted_by_cost = sorted(range(len(products)), key=lambda i: costs[i])
good_enough_2 = {}
remaining_capacity = capacity
cost_2 = 0
for idx in sorted_by_cost:
    units = min(demand[idx], remaining_capacity)
    good_enough_2[products[idx]] = {
        'Units': units,
        'Cost': costs[idx] * units,
        'Reason': 'Prioritize low-cost products'
    }
    cost_2 += costs[idx] * units
    remaining_capacity -= units

# Solution 3: Balanced approach (proportional to demand)
total_demand = sum(demand)
good_enough_3 = {}
cost_3 = 0
for i in range(len(products)):
    # Allocate capacity proportionally, but at least meet demand if possible
    proportional = int((demand[i] / total_demand) * capacity)
    units = max(demand[i], min(proportional, capacity - sum([good_enough_3.get(products[j], 0) for j in range(i)])))
    if sum([good_enough_3.get(products[j], 0) for j in range(i+1)]) > capacity:
        units = max(0, capacity - sum([good_enough_3.get(products[j], 0) for j in range(i)]))
    good_enough_3[products[i]] = {
        'Units': units,
        'Cost': costs[i] * units,
        'Reason': 'Balanced: proportional allocation'
    }
    cost_3 += costs[i] * units

# Create comparison
solutions = {
    'Optimal': optimal_solution,
    'Good Enough 1': good_enough_1,
    'Good Enough 2': good_enough_2,
    'Good Enough 3': good_enough_3
}

costs_all = {
    'Optimal': optimal_cost,
    'Good Enough 1': cost_1,
    'Good Enough 2': cost_2,
    'Good Enough 3': cost_3
}

print("ALL SOLUTIONS:")
print("=" * 70)
for name, sol in solutions.items():
    df = pd.DataFrame(sol).T
    print(f"\n{name}:")
    display(df)
    print(f"Total Cost: ${costs_all[name]:,.2f}")
    print(f"Difference from Optimal: ${costs_all[name] - optimal_cost:,.2f} ({((costs_all[name] - optimal_cost) / optimal_cost * 100):.1f}%)")


In [ ]:
comparison = pd.DataFrame({
    'Solution': ['Optimal', 'Good Enough 1', 'Good Enough 2', 'Good Enough 3'],
    'Total Cost ($)': [optimal_cost, cost_1, cost_2, cost_3],
    'Cost Difference from Optimal ($)': [0, cost_1 - optimal_cost, cost_2 - optimal_cost, cost_3 - optimal_cost],
    'Cost Difference (%)': [0, 
                            ((cost_1 - optimal_cost) / optimal_cost * 100),
                            ((cost_2 - optimal_cost) / optimal_cost * 100),
                            ((cost_3 - optimal_cost) / optimal_cost * 100)],
    'Implementation Difficulty': ['High (requires optimization)', 'Low (simple rule)', 'Low (simple rule)', 'Medium (calculation)']
})

print("COMPARISON: Optimal vs Good Enough")
print("=" * 70)
display(comparison.round(2))

print("\nKey Observations:")
print(f"  - Optimal solution has lowest cost: ${optimal_cost:,.2f}")
print(f"  - 'Good enough' solutions are {((max(cost_1, cost_2, cost_3) - optimal_cost) / optimal_cost * 100):.1f}% to {((min(cost_1, cost_2, cost_3) - optimal_cost) / optimal_cost * 100):.1f}% more expensive")
print(f"  - But 'good enough' solutions may be easier to implement or explain")
print(f"  - The small cost difference may not justify the complexity of optimization")


## Step 6: Visualize the Comparison

Let's visualize optimal vs "good enough" solutions:


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Cost comparison
ax1 = axes[0, 0]
solution_names = comparison['Solution'].values
costs_plot = comparison['Total Cost ($)'].values
colors = ['green' if name == 'Optimal' else 'orange' for name in solution_names]
bars1 = ax1.bar(solution_names, costs_plot, color=colors, edgecolor='black', linewidth=2)
ax1.set_ylabel('Total Cost ($)', fontsize=11, fontweight='bold')
ax1.set_title('Cost Comparison: Optimal vs Good Enough', fontsize=12, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)
for bar, cost in zip(bars1, costs_plot):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
             f'${cost:,.0f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# Plot 2: Cost difference from optimal
ax2 = axes[0, 1]
cost_diff = comparison['Cost Difference (%)'].values
bars2 = ax2.bar(solution_names, cost_diff, color=colors, edgecolor='black', linewidth=2)
ax2.set_ylabel('Cost Difference from Optimal (%)', fontsize=11, fontweight='bold')
ax2.set_title('How Close to Optimal?', fontsize=12, fontweight='bold')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax2.tick_params(axis='x', rotation=45)
ax2.grid(axis='y', alpha=0.3)
for bar, diff in zip(bars2, cost_diff):
    if diff > 0:
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                 f'+{diff:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=9)

# Plot 3: Production by product (optimal vs best good enough)
ax3 = axes[1, 0]
best_good_enough = min([('Good Enough 1', cost_1), ('Good Enough 2', cost_2), ('Good Enough 3', cost_3)], 
                        key=lambda x: x[1])[0]
x_pos = np.arange(len(products))
width = 0.35
optimal_units = [optimal_df.loc[p, 'Units'] for p in products]
good_enough_units = [pd.DataFrame(solutions[best_good_enough]).T.loc[p, 'Units'] for p in products]

ax3.bar(x_pos - width/2, optimal_units, width, label='Optimal', color='green', edgecolor='black')
ax3.bar(x_pos + width/2, good_enough_units, width, label=best_good_enough, color='orange', edgecolor='black')
ax3.set_xlabel('Product', fontsize=11, fontweight='bold')
ax3.set_ylabel('Units Produced', fontsize=11, fontweight='bold')
ax3.set_title(f'Production Levels: Optimal vs {best_good_enough}', fontsize=12, fontweight='bold')
ax3.set_xticks(x_pos)
ax3.set_xticklabels([p.replace('Product ', '') for p in products])
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# Plot 4: Implementation difficulty vs cost
ax4 = axes[1, 1]
difficulty_map = {'High (requires optimization)': 3, 'Medium (calculation)': 2, 'Low (simple rule)': 1}
difficulty = [difficulty_map[d] for d in comparison['Implementation Difficulty']]
scatter = ax4.scatter(difficulty, costs_plot, s=200, c=colors, edgecolors='black', linewidths=2, alpha=0.7)
for i, name in enumerate(solution_names):
    ax4.annotate(name, (difficulty[i], costs_plot[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=9)
ax4.set_xlabel('Implementation Difficulty (1=Low, 3=High)', fontsize=11, fontweight='bold')
ax4.set_ylabel('Total Cost ($)', fontsize=11, fontweight='bold')
ax4.set_title('Cost vs Implementation Difficulty', fontsize=12, fontweight='bold')
ax4.set_xticks([1, 2, 3])
ax4.set_xticklabels(['Low', 'Medium', 'High'])
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("  - Optimal solution has lowest cost but highest implementation difficulty")
print("  - 'Good enough' solutions are close to optimal but easier to implement")
print("  - Small cost differences may not justify optimization complexity")
print("  - Choose based on: cost difference, implementation difficulty, and context")


## Summary: "Good Enough" vs Optimal

**Optimal Solution**:
- Best solution according to objectives
- Found through systematic optimization
- Lowest cost (in this example)
- May be complex to implement or explain

**"Good Enough" Solution**:
- Nearly optimal (close to best)
- May be easier to implement
- May be easier to explain
- Small cost difference may not matter

**When to Choose Each**:
- **Choose optimal** when:
  - Cost differences are significant
  - Problem is high-stakes
  - Implementation complexity is acceptable
  - Optimization tools are available
  
- **Choose "good enough"** when:
  - Cost differences are small
  - Problem is low-stakes
  - Simplicity is valuable
  - Quick decisions are needed

**Practical Implication**:
- Optimal is best for objectives, but "good enough" may be best overall
- Consider implementation difficulty, robustness, and other factors
- Sometimes the best solution is the one you can actually implement
